In [ ]:
from summarizer import Summarizer

# Input text to be summarized
input_text = """
We are proud to build and release models that are industry-leading on both capabilities and safety.

More than a hundred million users and millions of developers rely on the work of our safety teams. We view safety as something we have to invest in and succeed at across multiple time horizons, from aligning today’s models to the far more capable systems we expect in the future. This work has always happened across OpenAI and our investment will only increase over time.

We believe in a balanced, scientific approach where safety measures are integrated into the development process from the outset. This ensures that our AI systems are both innovative and reliable, and can deliver benefits to society.

At today’s AI Seoul Summit, we're joining industry leaders, government officials, and members of civil society to discuss AI safety. While there is still more work to do, we are encouraged by the additional Frontier AI Safety Commitments that OpenAI and other companies agreed upon today. The Commitments call on companies to safely develop and deploy their frontier AI models while sharing information about their risk mitigation measures, aligning with steps we have already taken. These include a pledge to publish safety frameworks like the Preparedness Framework(opens in a new window) we developed and adopted last year. 

We are sharing 10 practices we actively use and improve upon. 
"""

# Create a BERT extractive summarizer
summarizer = Summarizer()

# Generate the summary
summary = summarizer(input_text, min_length=50, max_length=150)  # You can adjust the min_length and max_length parameters

# Output the summary
print("Original Text:")
print(input_text)
print("\nSummary:")
print(summary)